## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-07-23-16-00 +0000,wsj,President Trump reignited his global trade war...,https://www.wsj.com/economy/trade/trump-tariff...
1,2025-07-07-22-36-38 +0000,nyt,Haiti’s Landmark Oloffson Hotel is Destroyed i...,https://www.nytimes.com/2025/07/07/world/ameri...
2,2025-07-07-22-22-00 +0000,bbc,"Young campers, teachers and football coach amo...",https://www.bbc.com/news/articles/c5ygl8lpyyqo
3,2025-07-07-22-01-54 +0000,nyt,"As Renewed U.S. Tariffs Loom, Emerging Economi...",https://www.nytimes.com/2025/07/07/world/ameri...
4,2025-07-07-21-51-39 +0000,nyt,New Document Undermines Trump Administration’s...,https://www.nytimes.com/2025/07/07/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


C:\Users\CJ\AppData\Local\Temp\ipykernel_5232\2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,10
34,texas,6
4,trade,4
12,tariffs,4
64,countries,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
35,2025-07-07-10-57-00 +0000,wsj,President Trump is facing a pivotal time for r...,https://www.wsj.com/economy/trade/trump-faces-...,42
0,2025-07-07-23-16-00 +0000,wsj,President Trump reignited his global trade war...,https://www.wsj.com/economy/trade/trump-tariff...,39
12,2025-07-07-21-08-00 +0000,wsj,Here’s an annotated look at Trump’s tariff let...,https://www.wsj.com/politics/policy/read-an-an...,29
39,2025-07-07-05-17-39 +0000,bbc,Netanyahu visits US as Trump puts pressure to ...,https://www.bbc.com/news/articles/cy4ypze027ro,21
7,2025-07-07-21-43-18 +0000,bbc,Trump threatens tariffs on 14 countries from A...,https://www.bbc.com/news/articles/cd0vkl31085o,20


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
35,42,2025-07-07-10-57-00 +0000,wsj,President Trump is facing a pivotal time for r...,https://www.wsj.com/economy/trade/trump-faces-...
2,17,2025-07-07-22-22-00 +0000,bbc,"Young campers, teachers and football coach amo...",https://www.bbc.com/news/articles/c5ygl8lpyyqo
12,16,2025-07-07-21-08-00 +0000,wsj,Here’s an annotated look at Trump’s tariff let...,https://www.wsj.com/politics/policy/read-an-an...
0,14,2025-07-07-23-16-00 +0000,wsj,President Trump reignited his global trade war...,https://www.wsj.com/economy/trade/trump-tariff...
32,12,2025-07-07-13-06-32 +0000,bbc,South Africa's police minister accused of link...,https://www.bbc.com/news/articles/cx205ykppdjo
39,11,2025-07-07-05-17-39 +0000,bbc,Netanyahu visits US as Trump puts pressure to ...,https://www.bbc.com/news/articles/cy4ypze027ro
1,9,2025-07-07-22-36-38 +0000,nyt,Haiti’s Landmark Oloffson Hotel is Destroyed i...,https://www.nytimes.com/2025/07/07/world/ameri...
20,9,2025-07-07-17-42-19 +0000,bbc,US to remove Syria's HTS from list of foreign ...,https://www.bbc.com/news/articles/cj0m194v974o
16,9,2025-07-07-19-36-43 +0000,bbc,Former Tory cabinet minister Jones joins Refor...,https://www.bbc.com/news/articles/c93kwk00gkgo
40,9,2025-07-07-02-48-36 +0000,bbc,Government urged to keep education plans for c...,https://www.bbc.com/news/articles/cx2vn950d5go


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
